# PLEIADES Tutorial - SAMMY Exercise EX027: Generating Parameter Files Based on ENDF Files

This exercise demonstrates how to use SAMMY to generate parameter files starting from ENDF files.
The goal is to familiarize analysts with the process of running SAMMY directly from ENDF data and modifying the output for further analysis.

## Purpose
The purpose of this exercise is to learn how to:
1. Run SAMMY using ENDF files as input.
2. Generate SAMMY-style INPut and PARameter files for subsequent runs.
3. Modify and refine the generated files to fit experimental data.

## Description
SAMMY can read **File 2** of an **ENDF** file and perform a run as specified in the INPut file.
The output includes SAMMY-style INPut and PARameter files, which can be used for further analysis.
However, since ENDF files lack some information required by SAMMY, only simple no-Bayes runs can be performed directly from ENDF files.

## Instructions
1. **Initial Run**:
    - Use the command file `ex027a` to set up a no-Bayes run starting from the ENDF file.
    - The INPut file `ex027a.inp` does not include spin group information, which will be extracted from the ENDF file.
    - The experimental data file `ex027a.dat` assumes a 10% uncertainty due to the lack of provided uncertainties in the original data.
    - Run this setup to generate SAMMY-style INPut and PARameter files.

2. **Refining the Output**:
    - Modify the generated INPut and PARameter files for further analysis:
      - Update the thickness to `0.002800 atoms/barn`.
      - Enable single-scattering corrections and normalize as cross sections.
    - For Bayesian analysis, flag at least one parameter in the PARameter file.
    - Adjust the uncertainty settings in the INPut file by setting the `VMIN` parameter in Card Set 7 to account for unrealistic weights on background points.

3. **Fitting the Data**:
    - Start with small energy ranges for better fits, as the data set contains many points and gaps.
    - Normalize the data appropriately. For example, in the energy range from 6 to 100 eV, a normalization factor of nearly 10 is required.
    - Experiment with fitting strategies to improve the fit quality.

This exercise provides a hands-on approach to understanding the workflow of using SAMMY with ENDF files and refining the results for meaningful analysis.


## Step 1: Initial Run

Let's create a workspace directory to host all the runtime files for this exercise.

In [1]:
from pathlib import Path

# Get current working directory (notebook location)
current_dir = Path.cwd()

# Create a dedicated sammy_workspaces folder for all runs
sammy_base_dir = current_dir / "workspaces"
sammy_base_dir.mkdir(exist_ok=True)

# Create a working directory within our persistent folder
base_dir_name = f"sammy_endf_example"
working_dir = sammy_base_dir / base_dir_name  # This is for this tutorial
# -- fine grained sub-directories for different output
output_dir = working_dir / "results"
endf_dir = working_dir / "endf"
endf_output_dir = endf_dir / "results"
# -- create the directories
working_dir.mkdir(exist_ok=True)
output_dir.mkdir(exist_ok=True)
endf_dir.mkdir(exist_ok=True)
endf_output_dir.mkdir(exist_ok=True)

The example input, parameter and data files are located in `../samexm/ex027/`, so we will need to specify the source directory for easy access.

In [2]:
source_dir = current_dir / "../samexm/ex027"

# let's list the tree of the source directory
def list_tree(startpath: Path, prefix: str = ""):
    """Recursively print a tree structure of the directory like `tree` command."""
    entries = sorted(startpath.iterdir(), key=lambda x: (not x.is_dir(), x.name.lower()))
    entries_count = len(entries)

    for index, path in enumerate(entries):
        connector = "└── " if index == entries_count - 1 else "├── "
        print(prefix + connector + path.name)

        if path.is_dir():
            extension = "    " if index == entries_count - 1 else "│   "
            list_tree(path, prefix + extension)

print("Source directory tree:")
list_tree(source_dir)

Source directory tree:
├── endf
│   ├── results
│   │   ├── SAM46.DAT
│   │   ├── SAMMY.IO
│   │   ├── SAMMY.LPT
│   │   ├── SAMMY.LST
│   │   ├── SAMMY.ODF
│   │   ├── SAMMY.PLT
│   │   ├── SAMNDF.INP
│   │   ├── SAMNDF.PAR
│   │   ├── SAMQUA.PAR
│   │   └── SAMQUA.RED
│   ├── endf_dummy.dat
│   ├── ex027a
│   ├── ex027a.endf
│   └── ex027a.inp
├── ex027.dat
├── ex027b
├── ex027b.inp
├── ex027b.par
└── README.FIRST


Next step, we need to make sure that the `sammy` executable is available.

> Technically, PLEIADES SAMMY runner will perform the check to ensure that the executable is available, but it is a good practice to check it manually.

In [3]:
import subprocess

# Grab the SAMMY executable 
sammy_executable = subprocess.run(["which", "sammy"], capture_output=True, text=True).stdout.strip()

if not sammy_executable:
    raise RuntimeError("SAMMY executable not found. Please ensure SAMMY is installed and in your PATH.")

Assuming we found `sammy` in the `PATH` (i.e. no error in the cell above), we can configure a local SAMMY runner from PLEIADES.

In [4]:
from pleiades.sammy.factory import SammyFactory

runner = SammyFactory.create_runner(
    backend_type="local",
    working_dir=endf_dir,
    output_dir=endf_output_dir,
)

runner

2025-05-05 14:15:25 | DEBUG    | pleiades.sammy.factory:list_available_backends:76 - Local SAMMY found at: /Users/8cz/code.ornl.gov/SAMMY/build/bin/sammy
2025-05-05 14:15:26 | DEBUG    | pleiades.sammy.factory:list_available_backends:90 - Docker backend available
2025-05-05 14:15:26 | DEBUG    | pleiades.sammy.factory:list_available_backends:100 - NOVA credentials found


Now it's time to specify the input files for the run.

In [5]:
from pleiades.sammy.interface import SammyFiles

files_endf_step_0 = SammyFiles(
    input_file=source_dir / "endf/ex027a.inp",
    parameter_file=source_dir / "endf/ex027a.endf",
    data_file=source_dir / "endf/endf_dummy.dat",
)

Before we kick off the run, we need to allow sammy runner to check the file validity and move things into the working directory (if needed).

In [6]:
runner.prepare_environment(files=files_endf_step_0)

# Print the configuration to verify
print("Runner configuration:")
print(f"Working directory: {runner.config.working_dir}")
print(f"Output directory: {runner.config.output_dir}")
print(f"SAMMY executable: {runner.config.sammy_executable}")

2025-05-05 14:15:48 | DEBUG    | pleiades.sammy.backends.local:prepare_environment:44 - Validating input files
2025-05-05 14:15:48 | DEBUG    | pleiades.sammy.backends.local:prepare_environment:48 - Moving files to working directory
2025-05-05 14:15:48 | DEBUG    | pleiades.sammy.interface:move_to_working_dir:84 - Moving files to working directory: /Users/8cz/github.com/PLEIADES/examples/Notebooks/workspaces/sammy_endf_example/endf
2025-05-05 14:15:48 | DEBUG    | pleiades.sammy.interface:move_to_working_dir:100 - Copying input file: /Users/8cz/github.com/PLEIADES/examples/Notebooks/../samexm/ex027/endf/ex027a.inp -> /Users/8cz/github.com/PLEIADES/examples/Notebooks/workspaces/sammy_endf_example/endf/ex027a.inp
2025-05-05 14:15:48 | DEBUG    | pleiades.sammy.interface:move_to_working_dir:109 - Copying parameter file: /Users/8cz/github.com/PLEIADES/examples/Notebooks/../samexm/ex027/endf/ex027a.endf -> /Users/8cz/github.com/PLEIADES/examples/Notebooks/workspaces/sammy_endf_example/endf/

Runner configuration:
Working directory: /Users/8cz/github.com/PLEIADES/examples/Notebooks/workspaces/sammy_endf_example/endf
Output directory: /Users/8cz/github.com/PLEIADES/examples/Notebooks/workspaces/sammy_endf_example/endf/results
SAMMY executable: /Users/8cz/code.ornl.gov/SAMMY/build/bin/sammy


Now that everything is set up, we can run the SAMMY job.

In [7]:
# execute
result = runner.execute_sammy(files_endf_step_0)

# Print the results of the SAMMY run
if result.success:
    print(result.console_output)


2025-05-05 14:15:52 | INFO     | pleiades.sammy.backends.local:execute_sammy:62 - Starting SAMMY execution 16fa7ea3-b2dc-48c3-88ea-0ad642e03ed6
2025-05-05 14:15:52 | DEBUG    | pleiades.sammy.backends.local:execute_sammy:63 - Working directory: /Users/8cz/github.com/PLEIADES/examples/Notebooks/workspaces/sammy_endf_example/endf
2025-05-05 14:16:02 | INFO     | pleiades.sammy.backends.local:execute_sammy:93 - SAMMY execution completed successfully for 16fa7ea3-b2dc-48c3-88ea-0ad642e03ed6



 **********************************************************
 ***                                                    ***
 ***    SAMMY Version 8.1.pre-b1 - 57c3b4aa             ***                      
 ***                                                    ***
 **********************************************************

 *** SAMMAS       3 Jan 08 ***
                                         CPU for this section is      0.00 sec
 What is the name of the INPut file? 
 >>> ex027a.inp                                                             <<<
 What is the name of the PARameter file? 
 >>> ex027a.endf                                                            <<<
 What is the DATa file name? EMIN? EMAX? 
 >>> endf_dummy.dat                                                         <<<
 Emin and Emax =   1.000000E-05   10000.0    
 Skipping over unresolved range if reading ENDF data                  
ENDF file has Lrf=  3
 What is new EMIN? EMAX? DATa file name? 
 
                     

Notice that we have a lot of results file sitting in the working directory

In [8]:
list_tree(endf_dir)

├── results
├── endf_dummy.dat
├── ex027a.endf
├── ex027a.inp
├── SAM46.DAT
├── SAMMY.IO
├── SAMMY.LPT
├── SAMNDF.INP
├── SAMNDF.PAR
├── SAMQUA.PAR
└── SAMQUA.RED


Let's collect the SAMMY outputs and move them to the `output` directory, i.e. `results`

In [9]:
runner.collect_outputs(result=result)

2025-05-05 14:16:18 | INFO     | pleiades.sammy.interface:collect_outputs:306 - Collecting outputs for execution 16fa7ea3-b2dc-48c3-88ea-0ad642e03ed6
2025-05-05 14:16:18 | DEBUG    | pleiades.sammy.interface:collect_outputs:317 - Found known output file: SAMNDF.INP
2025-05-05 14:16:18 | DEBUG    | pleiades.sammy.interface:collect_outputs:317 - Found known output file: SAMNDF.PAR
2025-05-05 14:16:18 | DEBUG    | pleiades.sammy.interface:collect_outputs:317 - Found known output file: SAMMY.LPT
2025-05-05 14:16:18 | DEBUG    | pleiades.sammy.interface:collect_outputs:317 - Found known output file: SAMMY.IO
2025-05-05 14:16:18 | DEBUG    | pleiades.sammy.interface:collect_outputs:323 - Found additional output file: SAM46.DAT
2025-05-05 14:16:18 | DEBUG    | pleiades.sammy.interface:collect_outputs:323 - Found additional output file: SAMQUA.RED
2025-05-05 14:16:18 | DEBUG    | pleiades.sammy.interface:collect_outputs:323 - Found additional output file: SAMQUA.PAR
2025-05-05 14:16:18 | DEBUG

In [10]:
list_tree(endf_dir)

├── results
│   ├── SAM46.DAT
│   ├── SAMMY.IO
│   ├── SAMMY.LPT
│   ├── SAMNDF.INP
│   ├── SAMNDF.PAR
│   ├── SAMQUA.PAR
│   └── SAMQUA.RED
├── endf_dummy.dat
├── ex027a.endf
└── ex027a.inp


#### <span style="color:red;">The rest of the example will be made once the relevant classes for results and plotting are implemented in PLEIADES.</span>